In [62]:
from webapp import create_app
from vk_parser import *
import re


def preprocess_text(text):
    text = text.lower().replace('ё', 'е')
    text = text.replace('d', '')
    text = text.replace('rt', '')
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text)
    text = re.sub('@[^\s]+', '', text)
    text = re.sub(r'\b[a-zA-Zа-яА-Я1-9]\b', '', text)
    text = re.sub(r'[0-9]', '', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()


app = create_app()
with app.app_context():
    query = Comment.query.all()
    preprop_comm = []

    for comment in query:
        preprop_comm.append(preprocess_text(comment.comment_text))

In [3]:
preprop_comm

['так толсто что аж тонко было весело',
 'мне кажется или это пародия на мойти пайтона',
 'типичная осада железных войнов крепостей имперских кулаков',
 'просто еще немного унижения железных плакс',
 '',
 'я даже не удивлен',
 'если кому интересно и если кто еще не читал классно осады жв описаны у грэма макнилла в омнибусе железные воины про полукровку хонсю',
 'хочу еще',
 'ааа есть перевод',
 'юмор всем полком смеемся',
 'ересь',
 'госпожа комиссар бдит за дисциплиной',
 'очень практичный шланг в голову входит это дерьмо вкачивать или выкачивать',
 'какие крылья атасные у ребят',
 'мм лоялам теперь тоже шланги в ухо положены для подачи варп даста',
 'друг просит фулы братцы грозит инквизиции пожаловаться',
 'это нурглячья пиявка атакуе',
 'too hot',
 'а что сейчас с ангелами за движ пошел',
 'зная художника есть версия и с членом',
 'блютуз гарнитура походу',
 'одобрено инквизицией',
 'где одмин волколюб что он может сказать за свой саплай',
 'выпускной в схоле',
 '1 к лояльности',
 

In [21]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kamikaze666\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kamikaze666\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

In [8]:
import pandas as pd
import numpy as np

# Считываем данные
n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_positive = pd.read_csv('training_data/positive.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])
data_negative = pd.read_csv('training_data/negative.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])

# Формируем сбалансированный датасет
sample_size = min(data_positive.shape[0], data_negative.shape[0])
raw_data = np.concatenate((data_positive['text'].values[:sample_size],
                           data_negative['text'].values[:sample_size]), axis=0)
labels = [1] * sample_size + [0] * sample_size

In [9]:
data_positive

,text
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"Да, все-таки он немного похож на него. Но мой ..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3,"RT @digger2912: ""Кто то в углу сидит и погибае..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...
...,...
114906,"Спала в родительском доме, на своей кровати......"
114907,RT @jebesilofyt: Эх... Мы немного решили сокра...
114908,"Что происходит со мной, когда в эфире #proacti..."
114909,"""Любимая,я подарю тебе эту звезду..."" Имя како..."


In [10]:
data_negative

,text
0,на работе был полный пиддес :| и так каждое за...
1,"Коллеги сидят рубятся в Urban terror, а я из-з..."
2,@elina_4post как говорят обещаного три года жд...
3,"Желаю хорошего полёта и удачной посадки,я буду..."
4,"Обновил за каким-то лешим surf, теперь не рабо..."
...,...
111918,Но не каждый хочет что то исправлять:( http://...
111919,скучаю так :-( только @taaannyaaa вправляет мо...
111920,"Вот и в школу, в говно это идти уже надо("
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :..."


In [63]:
data = [preprocess_text(t) for t in raw_data]

In [51]:
raw_data

array(['@first_timee хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)',
       'Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D',
       'RT @KatiaCheh: Ну ты идиотка) я испугалась за тебя!!!', ...,
       'Вот и в школу, в говно это идти уже надо(',
       'RT @_Them__: @LisaBeroud Тауриэль, не грусти :( *обнял*',
       'Такси везет меня на работу. Раздумываю приплатить, чтобы меня втащили на пятый этаж. Лифта то нет :('],
      dtype=object)

In [41]:
data

['хоть я и школота но поверь у нас то же самое d общество профилирующий предмет типа',
 'да все таки он немного похож на него но мой мальчик все равно лучше d',
 'rt ну ты идиотка я испугалась за тебя',
 'rt кто то в углу сидит и погибает от голода а мы еще 2 порции взяли хотя уже и так жрать не хотим dd',
 'вот что значит страшилка d но блин посмотрев все части у тебя создастся ощущение что авторы курили что то d',
 'ну любишь или нет я не знаю кто ты бля d',
 'rt ох 9 d ну это конечно же чтобы у нее было много друзей ведь она такая мимими lt 3',
 'rt у тебя есть ухажер нет мои уши не кто не жрет d',
 'поприветствуем моего нового читателя',
 'теперь у меня есть частичка сиднея sydney sydneyoperahouse',
 'люблю маму и папу а в остальное я так влюбляюсь привязываюсь',
 'rt как то я забыла что вчера приехал из деревни наш котэня и испугалась когда он утром пришел ко мне общаться доброе у',
 'так было задумано да',
 'дааааа ты ты ты и только ты',
 'о нет вы ведь все равно обзоры урезаете 

In [64]:
data

['хоть школота но поверь нас то же самое общество профилирующий предмет типа',
 'да все таки он немного похож на него но мой мальчик все равно лучше',
 'ну ты идиотка испугалась за тебя',
 'кто то углу сидит погибает от голода мы еще порции взяли хотя уже так жрать не хотим',
 'вот что значит страшилка но блин посмотрев все части тебя создастся ощущение что авторы курили что то',
 'ну любишь или нет не знаю кто ты бля',
 'ох ну это конечно же чтобы нее было много друзей ведь она такая мимими lt',
 'тебя есть ухажер нет мои уши не кто не жрет',
 'поприветствуем моего нового читателя',
 'теперь меня есть частичка сиднея syney syneyoperahouse',
 'люблю маму папу остальное так влюбляюсь привязываюсь',
 'как то забыла что вчера приехал из деревни наш котэня испугалась когда он утром пришел ко мне общаться доброе',
 'так было задумано да',
 'дааааа ты ты ты только ты',
 'нет вы ведь все равно обзоры урезаете просто одна версия сильно порезанная другая полная только без мусора',
 'смотри там 

In [23]:
stop_words = set(stopwords.words("russian"))

In [32]:
words = [word_tokenize(sent) for sent in data]

In [38]:
words

[['USER',
  'хоть',
  'я',
  'и',
  'школота',
  'но',
  'поверь',
  'у',
  'нас',
  'то',
  'же',
  'самое',
  'd',
  'общество',
  'профилирующий',
  'предмет',
  'типа'],
 ['да',
  'все',
  'таки',
  'он',
  'немного',
  'похож',
  'на',
  'него',
  'но',
  'мой',
  'мальчик',
  'все',
  'равно',
  'лучше',
  'd'],
 ['rt', 'USER', 'ну', 'ты', 'идиотка', 'я', 'испугалась', 'за', 'тебя'],
 ['rt',
  'USER',
  'кто',
  'то',
  'в',
  'углу',
  'сидит',
  'и',
  'погибает',
  'от',
  'голода',
  'а',
  'мы',
  'еще',
  '2',
  'порции',
  'взяли',
  'хотя',
  'уже',
  'и',
  'так',
  'жрать',
  'не',
  'хотим',
  'dd',
  'URL'],
 ['USER',
  'вот',
  'что',
  'значит',
  'страшилка',
  'd',
  'но',
  'блин',
  'посмотрев',
  'все',
  'части',
  'у',
  'тебя',
  'создастся',
  'ощущение',
  'что',
  'авторы',
  'курили',
  'что',
  'то',
  'd'],
 ['ну',
  'любишь',
  'или',
  'нет',
  'я',
  'не',
  'знаю',
  'кто',
  'ты',
  'бля',
  'd',
  'URL'],
 ['rt',
  'USER',
  'ох',
  '9',
  'd',
 

In [37]:
stop_words

{'а',
 'без',
 'более',
 'больше',
 'будет',
 'будто',
 'бы',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'вам',
 'вас',
 'вдруг',
 'ведь',
 'во',
 'вот',
 'впрочем',
 'все',
 'всегда',
 'всего',
 'всех',
 'всю',
 'вы',
 'где',
 'да',
 'даже',
 'два',
 'для',
 'до',
 'другой',
 'его',
 'ее',
 'ей',
 'ему',
 'если',
 'есть',
 'еще',
 'ж',
 'же',
 'за',
 'зачем',
 'здесь',
 'и',
 'из',
 'или',
 'им',
 'иногда',
 'их',
 'к',
 'как',
 'какая',
 'какой',
 'когда',
 'конечно',
 'кто',
 'куда',
 'ли',
 'лучше',
 'между',
 'меня',
 'мне',
 'много',
 'может',
 'можно',
 'мой',
 'моя',
 'мы',
 'на',
 'над',
 'надо',
 'наконец',
 'нас',
 'не',
 'него',
 'нее',
 'ней',
 'нельзя',
 'нет',
 'ни',
 'нибудь',
 'никогда',
 'ним',
 'них',
 'ничего',
 'но',
 'ну',
 'о',
 'об',
 'один',
 'он',
 'она',
 'они',
 'опять',
 'от',
 'перед',
 'по',
 'под',
 'после',
 'потом',
 'потому',
 'почти',
 'при',
 'про',
 'раз',
 'разве',
 'с',
 'сам',
 'свою',
 'себе',
 'себя',
 'сейчас',
 'со',
 'совсем',
 'так

In [34]:
words_filtered = []
for w in words:
    if w not in stop_words:
        words_filtered.append(w)

TypeError: unhashable type: 'list'